In [137]:
import os
import json
from csv import DictReader, DictWriter
import nltk.sentiment.util as nlh
import numpy as np
from numpy import array
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.metrics import accuracy_score
from nltk.sentiment import SentimentIntensityAnalyzer
from time import time
def polarity(text):
    """
    Output polarity scores for a text using Vader approach.

    :param text: a text whose polarity has to be evaluated.
    """
    
    vader_analyzer = SentimentIntensityAnalyzer()
    return (vader_analyzer.polarity_scores(text))


SEED = 5

'''
The ItemSelector class was created by Matt Terry to help with using
Feature Unions on Heterogeneous Data Sources

All credit goes to Matt Terry for the ItemSelector class below

For more information:
http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
'''
class ItemSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


"""
This is an example of a custom feature transformer. The constructor is used
to store the state (e.g like if you need to store certain words/vocab), the
fit method is used to update the state based on the training data, and the
transform method is used to transform the data into the new feature(s). In
this example, we simply use the length of the movie review as a feature. This
requires no state, so the constructor and fit method do nothing.
"""
class CapsTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples), 1))
        i = 0
        for ex in examples:
            features[i, 0] = sum(1 for c in ex if c.isupper())
            i += 1
        #print (features)
        return features

class TextLengthTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples), 1))
        i = 0
        for ex in examples:
            features[i, 0] = len(ex)
            i += 1
        #print (features)
        return features
class NumberSentencesTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples), 1))
        i = 0
        for ex in examples:
            features[i, 0] = ex.count('.')
            i += 1
        #print (features)
        return features

class AvgWordLengthTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples), 1))
        i = 0
        for ex in examples:
            features[i, 0] = np.mean([len(word) for word in ex.split()])
            i += 1
        #print (features)
        return features

class PosTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:
            
            n = polarity(ex)
            
            #print ("Here:"+str(n))
            
            features[i,0] = n['pos']
            i += 1
        #print (features)
        return features

class NegTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:

            n = polarity(ex)

            #print ("Here:"+str(n))
            
            features[i,0] = n['neg']
            i += 1
        #print (features)
        return features

class NeutralTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        for ex in examples:

            n = polarity(ex)

            #print ("Here:"+str(n))
            
            features[i,0] = n['neu']
            i += 1
        #print (features)
        return features

class ExclamationTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        count = 0
        for ex in examples:
            #print (ex)
            count = 0
            if ('!' in ex.split()):
                count +=1

            #print ("Here:"+str(n))

            features[i,0] = count
            i += 1
        #print (features)
        return features

class QuotedTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, examples):
        return self

    def transform(self, examples):
        features = np.zeros((len(examples),1))
        i = 0
        count = 0
        for ex in examples:
            #print (ex)
            count = 0
            if ('\"' in ex.split()):
                count +=1

            #print ("Here:"+str(n))

            features[i,0] = count
            i += 1
        #print (features)
        return features


# TODO: Add custom feature transformers for the movie review data

class Featurizer:
    def __init__(self):
        # To add new features, just add a new pipeline to the feature union
        # The ItemSelector is used to select certain pieces of the input data
        # In this case, we are selecting the plaintext of the input data

        # TODO: Add any new feature transformers or other features to the FeatureUnion
        self.all_features = FeatureUnion([
            ('text_length', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('text_length', TextLengthTransformer()),
            
            ])),
            ('exclamation', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('exclamation', ExclamationTransformer()),
            ])),
            ('quotes', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('quotes', QuotedTransformer()),
            ])),
            ('numbersentences', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('numbersentences', NumberSentencesTransformer()),
            ])),
            ('avgwordlength', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('avgwordlength', AvgWordLengthTransformer()),
            ])),
            ('positive', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('positive', PosTransformer()),
            ])),
            ('negative', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('negative', NegTransformer()),
            ])),
            ('caps', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('caps', CapsTransformer()),
            ])),


            ('neutral', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('neutral', NeutralTransformer()),
            ])),            
            ('grams', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('grams', CountVectorizer(stop_words='english', ngram_range=(1,2))),

            ])),
            ('tfidf', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf', TfidfVectorizer()),
            ])), 
           
        ])

    def train_feature(self, examples):
        return self.all_features.fit_transform(examples)

    def test_feature(self, examples):
        return self.all_features.transform(examples)

if __name__ == "__main__":

    # Read in data

    # Split dataset
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    f = open('trainf.txt',"r")
    for re in f.readlines():
        review = re.split('\t')[1]
        #print ("gennnn")
        X_train.append(review)
        y_train.append(0)
    t = open('traint.txt',"r")
    for re in t.readlines():
        review = re.split('\t')[1]
        #print ("soooop")
        X_train.append(review)
        y_train.append(1)
    f = open('testf.txt',"r")
    for re in f.readlines():
        #print ("negggg")
        review = re.split('\t')[1]
        X_test.append(review)
        y_test.append(0)
    t = open('testt.txt',"r")
    for re in t.readlines():
        review = re.split('\t')[1]
        #print ("posss")
        X_test.append(review)
        y_test.append(1)
    #print ("here:")
    #print(y_train)
    #print(y_test)

    #X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=SEED)

    feat = Featurizer()

    labels = []
    for l in y_train:
        if not l in labels:
            labels.append(l)

    print("Label set: %s\n" % str(labels))

    # Here we collect the train features
    # The inner dictionary contains certain pieces of the input data that we
    # would like to be able to select with the ItemSelector
    # The text key refers to the plaintext


    feat_train = feat.train_feature({
        'text': [t for t in X_train]
    })
    # Here we collect the test features
    feat_test = feat.test_feature({
        'text': [t for t in X_test]
    })

    #print(feat_train)
    #print(set(y_train))

    # Train classifier
    lr = SGDClassifier(loss='log',penalty='l2',alpha=0.0001,max_iter=15000, shuffle=True,verbose=2)
    lr.fit(feat_train,y_train)
    y_pred = lr.predict(feat_train)
    accuracy = accuracy_score(y_pred,y_train)
    print ("accuracy on train:")
    print (accuracy)
    y_pred = lr.predict(feat_test)
    accuracy = accuracy_score(y_pred,y_test)
    print ("accuracy on test:")
    print (accuracy)

    # EXTRA CREDIT: Replace the following code with scikit-learn cross validation
    # and determine the best 'alpha' parameter for regularization in the SGDClassifier


Label set: [0, 1]

-- Epoch 1
Norm: 3601.35, NNZs: 12730, Bias: -0.304211, T: 321, Avg. loss: 1219783.686809
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 2375.75, NNZs: 16573, Bias: -0.547911, T: 642, Avg. loss: 885118.505416
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 1940.34, NNZs: 18472, Bias: -1.051688, T: 963, Avg. loss: 676462.894360
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 2623.47, NNZs: 19343, Bias: -0.987284, T: 1284, Avg. loss: 546296.261605
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 2779.49, NNZs: 20198, Bias: -0.869315, T: 1605, Avg. loss: 612771.048570
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 2977.81, NNZs: 20357, Bias: -0.829626, T: 1926, Avg. loss: 393796.280922
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 2390.64, NNZs: 20400, Bias: -1.033677, T: 2247, Avg. loss: 473318.394919
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 2697.91, NNZs: 20593, Bias: -0.881876, T: 2568, Avg. loss: 430158.838803
Total training tim

-- Epoch 452
Norm: 2665.01, NNZs: 20700, Bias: -3.973794, T: 145092, Avg. loss: 4580.269822
Total training time: 0.20 seconds.
-- Epoch 453
Norm: 2664.05, NNZs: 20700, Bias: -3.974475, T: 145413, Avg. loss: 6231.561467
Total training time: 0.20 seconds.
-- Epoch 454
Norm: 2663.53, NNZs: 20700, Bias: -3.975160, T: 145734, Avg. loss: 5886.727723
Total training time: 0.20 seconds.
-- Epoch 455
Norm: 2663.26, NNZs: 20700, Bias: -3.975025, T: 146055, Avg. loss: 5473.672262
Total training time: 0.20 seconds.
-- Epoch 456
Norm: 2662.09, NNZs: 20700, Bias: -3.973662, T: 146376, Avg. loss: 6447.819934
Total training time: 0.20 seconds.
-- Epoch 457
Norm: 2662.99, NNZs: 20700, Bias: -3.971551, T: 146697, Avg. loss: 8573.418411
Total training time: 0.20 seconds.
-- Epoch 458
Norm: 2663.02, NNZs: 20700, Bias: -3.972224, T: 147018, Avg. loss: 6437.952582
Total training time: 0.20 seconds.
-- Epoch 459
Norm: 2663.06, NNZs: 20700, Bias: -3.966824, T: 147339, Avg. loss: 7254.224879
Total training time

Norm: 2272.49, NNZs: 20700, Bias: -4.134701, T: 291468, Avg. loss: 1801.792533
Total training time: 0.40 seconds.
-- Epoch 909
Norm: 2272.08, NNZs: 20700, Bias: -4.133879, T: 291789, Avg. loss: 2187.823523
Total training time: 0.40 seconds.
-- Epoch 910
Norm: 2271.16, NNZs: 20700, Bias: -4.136611, T: 292110, Avg. loss: 1997.422945
Total training time: 0.40 seconds.
-- Epoch 911
Norm: 2270.65, NNZs: 20700, Bias: -4.137483, T: 292431, Avg. loss: 1919.193868
Total training time: 0.40 seconds.
-- Epoch 912
Norm: 2269.52, NNZs: 20700, Bias: -4.137483, T: 292752, Avg. loss: 1865.933780
Total training time: 0.41 seconds.
-- Epoch 913
Norm: 2268.97, NNZs: 20700, Bias: -4.139183, T: 293073, Avg. loss: 2411.275162
Total training time: 0.41 seconds.
-- Epoch 914
Norm: 2267.93, NNZs: 20700, Bias: -4.139864, T: 293394, Avg. loss: 1971.443727
Total training time: 0.41 seconds.
-- Epoch 915
Norm: 2267.08, NNZs: 20700, Bias: -4.138845, T: 293715, Avg. loss: 2079.803400
Total training time: 0.41 second

Norm: 1843.42, NNZs: 20700, Bias: -4.187863, T: 433029, Avg. loss: 985.387846
Total training time: 0.60 seconds.
-- Epoch 1350
Norm: 1842.68, NNZs: 20700, Bias: -4.187863, T: 433350, Avg. loss: 480.640524
Total training time: 0.61 seconds.
-- Epoch 1351
Norm: 1841.80, NNZs: 20700, Bias: -4.187863, T: 433671, Avg. loss: 582.876306
Total training time: 0.61 seconds.
-- Epoch 1352
Norm: 1840.69, NNZs: 20700, Bias: -4.187863, T: 433992, Avg. loss: 311.365646
Total training time: 0.61 seconds.
-- Epoch 1353
Norm: 1839.47, NNZs: 20700, Bias: -4.188093, T: 434313, Avg. loss: 131.696208
Total training time: 0.61 seconds.
-- Epoch 1354
Norm: 1838.45, NNZs: 20700, Bias: -4.188353, T: 434634, Avg. loss: 319.165346
Total training time: 0.61 seconds.
-- Epoch 1355
Norm: 1837.41, NNZs: 20700, Bias: -4.187665, T: 434955, Avg. loss: 336.497604
Total training time: 0.61 seconds.
-- Epoch 1356
Norm: 1836.51, NNZs: 20700, Bias: -4.188325, T: 435276, Avg. loss: 460.289404
Total training time: 0.61 seconds

Norm: 1452.79, NNZs: 20700, Bias: -4.191693, T: 554688, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1729
Norm: 1451.95, NNZs: 20700, Bias: -4.191693, T: 555009, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1730
Norm: 1451.11, NNZs: 20700, Bias: -4.191693, T: 555330, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1731
Norm: 1450.28, NNZs: 20700, Bias: -4.191693, T: 555651, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1732
Norm: 1449.44, NNZs: 20700, Bias: -4.191693, T: 555972, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1733
Norm: 1448.61, NNZs: 20700, Bias: -4.191693, T: 556293, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1734
Norm: 1447.77, NNZs: 20700, Bias: -4.191693, T: 556614, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1735
Norm: 1446.94, NNZs: 20700, Bias: -4.191693, T: 556935, Avg. loss: 0.000000
Total training time: 0.81 seconds.
-- Epoch 1736


...

Norm: 195.73, NNZs: 20700, Bias: -4.189692, T: 4193544, Avg. loss: 0.000046
Total training time: 7.05 seconds.
-- Epoch 13065
Norm: 195.72, NNZs: 20700, Bias: -4.189692, T: 4193865, Avg. loss: 0.000045
Total training time: 7.05 seconds.
-- Epoch 13066
Norm: 195.70, NNZs: 20700, Bias: -4.189692, T: 4194186, Avg. loss: 0.000046
Total training time: 7.05 seconds.
-- Epoch 13067
Norm: 195.69, NNZs: 20700, Bias: -4.189692, T: 4194507, Avg. loss: 0.000045
Total training time: 7.05 seconds.
-- Epoch 13068
Norm: 195.67, NNZs: 20700, Bias: -4.189691, T: 4194828, Avg. loss: 0.000044
Total training time: 7.05 seconds.
-- Epoch 13069
Norm: 195.66, NNZs: 20700, Bias: -4.189691, T: 4195149, Avg. loss: 0.000045
Total training time: 7.06 seconds.
-- Epoch 13070
Norm: 195.64, NNZs: 20700, Bias: -4.189691, T: 4195470, Avg. loss: 0.000047
Total training time: 7.06 seconds.
-- Epoch 13071
Norm: 195.63, NNZs: 20700, Bias: -4.189691, T: 4195791, Avg. loss: 0.000047
Total training time: 7.06 seconds.
-- Epoc

-- Epoch 13475
Norm: 189.76, NNZs: 20700, Bias: -4.189628, T: 4325475, Avg. loss: 0.000050
Total training time: 7.25 seconds.
-- Epoch 13476
Norm: 189.75, NNZs: 20700, Bias: -4.189628, T: 4325796, Avg. loss: 0.000049
Total training time: 7.25 seconds.
-- Epoch 13477
Norm: 189.74, NNZs: 20700, Bias: -4.189628, T: 4326117, Avg. loss: 0.000046
Total training time: 7.26 seconds.
-- Epoch 13478
Norm: 189.72, NNZs: 20700, Bias: -4.189627, T: 4326438, Avg. loss: 0.000048
Total training time: 7.26 seconds.
-- Epoch 13479
Norm: 189.71, NNZs: 20700, Bias: -4.189627, T: 4326759, Avg. loss: 0.000050
Total training time: 7.26 seconds.
-- Epoch 13480
Norm: 189.69, NNZs: 20700, Bias: -4.189627, T: 4327080, Avg. loss: 0.000049
Total training time: 7.26 seconds.
-- Epoch 13481
Norm: 189.68, NNZs: 20700, Bias: -4.189627, T: 4327401, Avg. loss: 0.000049
Total training time: 7.26 seconds.
-- Epoch 13482
Norm: 189.67, NNZs: 20700, Bias: -4.189627, T: 4327722, Avg. loss: 0.000050
Total training time: 7.26 s

-- Epoch 13886
Norm: 184.15, NNZs: 20700, Bias: -4.189562, T: 4457406, Avg. loss: 0.000052
Total training time: 7.46 seconds.
-- Epoch 13887
Norm: 184.14, NNZs: 20700, Bias: -4.189562, T: 4457727, Avg. loss: 0.000051
Total training time: 7.46 seconds.
-- Epoch 13888
Norm: 184.12, NNZs: 20700, Bias: -4.189562, T: 4458048, Avg. loss: 0.000052
Total training time: 7.46 seconds.
-- Epoch 13889
Norm: 184.11, NNZs: 20700, Bias: -4.189562, T: 4458369, Avg. loss: 0.000050
Total training time: 7.46 seconds.
-- Epoch 13890
Norm: 184.10, NNZs: 20700, Bias: -4.189562, T: 4458690, Avg. loss: 0.000052
Total training time: 7.46 seconds.
-- Epoch 13891
Norm: 184.08, NNZs: 20700, Bias: -4.189562, T: 4459011, Avg. loss: 0.000051
Total training time: 7.46 seconds.
-- Epoch 13892
Norm: 184.07, NNZs: 20700, Bias: -4.189561, T: 4459332, Avg. loss: 0.000051
Total training time: 7.46 seconds.
-- Epoch 13893
Norm: 184.06, NNZs: 20700, Bias: -4.189561, T: 4459653, Avg. loss: 0.000051
Total training time: 7.46 s

Norm: 178.83, NNZs: 20700, Bias: -4.189497, T: 4589979, Avg. loss: 0.000055
Total training time: 7.66 seconds.
-- Epoch 14300
Norm: 178.82, NNZs: 20700, Bias: -4.189496, T: 4590300, Avg. loss: 0.000051
Total training time: 7.66 seconds.
-- Epoch 14301
Norm: 178.81, NNZs: 20700, Bias: -4.189496, T: 4590621, Avg. loss: 0.000053
Total training time: 7.66 seconds.
-- Epoch 14302
Norm: 178.79, NNZs: 20700, Bias: -4.189496, T: 4590942, Avg. loss: 0.000057
Total training time: 7.66 seconds.
-- Epoch 14303
Norm: 178.78, NNZs: 20700, Bias: -4.189496, T: 4591263, Avg. loss: 0.000055
Total training time: 7.66 seconds.
-- Epoch 14304
Norm: 178.77, NNZs: 20700, Bias: -4.189496, T: 4591584, Avg. loss: 0.000054
Total training time: 7.66 seconds.
-- Epoch 14305
Norm: 178.76, NNZs: 20700, Bias: -4.189496, T: 4591905, Avg. loss: 0.000054
Total training time: 7.66 seconds.
-- Epoch 14306
Norm: 178.74, NNZs: 20700, Bias: -4.189495, T: 4592226, Avg. loss: 0.000054
Total training time: 7.66 seconds.
-- Epoc

Norm: 174.24, NNZs: 20700, Bias: -4.189436, T: 4710996, Avg. loss: 0.000055
Total training time: 7.86 seconds.
-- Epoch 14677
Norm: 174.23, NNZs: 20700, Bias: -4.189436, T: 4711317, Avg. loss: 0.000056
Total training time: 7.86 seconds.
-- Epoch 14678
Norm: 174.22, NNZs: 20700, Bias: -4.189436, T: 4711638, Avg. loss: 0.000055
Total training time: 7.86 seconds.
-- Epoch 14679
Norm: 174.20, NNZs: 20700, Bias: -4.189436, T: 4711959, Avg. loss: 0.000056
Total training time: 7.86 seconds.
-- Epoch 14680
Norm: 174.19, NNZs: 20700, Bias: -4.189436, T: 4712280, Avg. loss: 0.000055
Total training time: 7.86 seconds.
-- Epoch 14681
Norm: 174.18, NNZs: 20700, Bias: -4.189436, T: 4712601, Avg. loss: 0.000056
Total training time: 7.86 seconds.
-- Epoch 14682
Norm: 174.17, NNZs: 20700, Bias: -4.189436, T: 4712922, Avg. loss: 0.000056
Total training time: 7.86 seconds.
-- Epoch 14683
Norm: 174.16, NNZs: 20700, Bias: -4.189435, T: 4713243, Avg. loss: 0.000056
Total training time: 7.86 seconds.
-- Epoc

In [83]:
from sklearn.svm import *

In [158]:
feat_test=normalize(feat_test,axis=1)

In [157]:
feat_train=normalize(feat_train,axis=1)

In [166]:
lr=SVC(C=0.0025, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=3.0, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.01, verbose=False)#lr=LinearSVC()
lr.fit(feat_train,yw)

SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=3.0, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.01, verbose=False)

In [141]:
yw=[]
wds2=[]
c=0
for i in X_train:
    wds2.append(i)
    w3=i.split('.')
    yw.append(y_train[c])
    for j in w3:
        wds2.append(i)
        yw.append(y_train[c])
    c+=1

In [165]:
feat = Featurizer()
feat_train = feat.train_feature({
        'text': [t for t in wds2]
    })
    # Here we collect the test features
feat_test = feat.test_feature({
        'text': [t for t in X_test]
    })

In [154]:
from sklearn.preprocessing import normalize

In [167]:
lr.score(feat_train,yw)

1.0

In [168]:
lr.score(feat_test,y_test)

0.51401869158878499

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


In [152]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]


In [153]:
for i in classifiers:
    print (i)
    try:
        wds=i.fit(feat_train,yw)
        print (i.score(feat_train,yw),i.score(feat_test,y_test))
    except:
        continue

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')
1.0 0.504672897196
SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
1.0 0.514018691589
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
1.0 0.495327102804
GaussianProcessClassifier(copy_X_train=True,
             kernel=1**2 * RBF(length_scale=1), max_iter_predict=100,
             multi_class='one_vs_rest', n_jobs=1, n_restarts_optimizer=0,
             optimizer='fmin_l_bfgs_b', random_state=None,
             warm_start=False)
DecisionTreeClassifier(class_weight=None, criterion=

In [50]:
i.kernel

1**2 * RBF(length_scale=1)